# Chapter 2: Routing

## Hands-On Code Example (LangChain)

> Adapted and modified from https://docs.google.com/document/d/1RZ5-2fykDQKOBx01pwfKkDe0GCs5ydca7xW9Q4wqS_M/edit?tab=t.0
> 
> Mi  8 Okt 2025 15:57:10 BST

This Python code defines an AI-powered crew using the CrewAI framework to generate a blog post about AI trends. It starts by setting up the environment, loading API keys from a .env file. The core of the application involves defining two agents: a researcher to find and summarize AI trends, and a writer to create a blog post based on the research. 

Two tasks are defined accordingly: one for researching the trends and another for writing the blog post, with the writing task depending on the output of the research task. These agents and tasks are then assembled into a Crew, specifying a sequential process where tasks are executed in order. The Crew is initialized with the agents, tasks, and a language model (specifically the "gemini-2.0-flash" model). The main function executes this crew using the kickoff() method, orchestrating the collaboration between the agents to produce the desired output. Finally, the code prints the final result of the crew's execution, which is the generated blog post.

In [1]:
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
def setup_environment():
   """Loads environment variables and checks for the required API key."""
   load_dotenv()
   if not os.getenv("GOOGLE_API_KEY"):
       raise ValueError("GOOGLE_API_KEY not found. Please set it in your .env file.")
   if not os.getenv("OPENAI_API_KEY"):
       raise ValueError("OPENAI_API_KEY not found. Please set it in your .env file.")

**Notes**: 

Crew seems to need an OpenAI API, so I added the code: 

```python 
   if not os.getenv("OPENAI_API_KEY"):
       raise ValueError("OPENAI_API_KEY not found. Please set it in your .env file.")
```

**Why CrewAI Needs OpenAI API Key**

CrewAI has a dependency on OpenAI's API for several internal operations, even when you specify a different LLM like Gemini. Here are the main reasons:

1. Default LLM Fallback
CrewAI uses OpenAI's models as the default language model for various internal operations. Even when you specify a different LLM (like Gemini) for your main tasks, CrewAI may still fall back to OpenAI for certain operations.

2. Agent Reasoning and Planning
CrewAI uses OpenAI models for:
    - Agent reasoning and decision-making
    - Task planning and coordination
    - Internal dialogue between agents
    - Error handling and recovery

3. Tool Usage and Function Calling
CrewAI relies on OpenAI's function calling capabilities for:
    - Tool selection and execution
    - Parameter extraction
    - Result processing

4. Embeddings and Memory
Some CrewAI features use OpenAI for:
    - Creating embeddings for memory systems
    - Semantic search within agent context
    - Knowledge retrieval

In [3]:
def main():
   """
   Initializes and runs the AI crew for content creation using the latest Gemini model.
   """
   setup_environment()

   # Define the language model to use.
   # Updated to a model from the Gemini 2.0 series for better performance and features.
   # For cutting-edge (preview) capabilities, you could use "gemini-2.5-flash".
   llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

   # Define Agents with specific roles and goals
   researcher = Agent(
       role='Senior Research Analyst',
       goal='Find and summarize the latest trends in AI.',
       backstory="You are an experienced research analyst with a knack for identifying key trends and synthesizing information.",
       verbose=True,
       allow_delegation=False,
   )

   writer = Agent(
       role='Technical Content Writer',
       goal='Write a clear and engaging blog post based on research findings.',
       backstory="You are a skilled writer who can translate complex technical topics into accessible content.",
       verbose=True,
       allow_delegation=False,
   )

   # Define Tasks for the agents
   research_task = Task(
       description="Research the top 3 emerging trends in Artificial Intelligence in 2024-2025. Focus on practical applications and potential impact.",
       expected_output="A detailed summary of the top 3 AI trends, including key points and sources.",
       agent=researcher,
   )

   writing_task = Task(
       description="Write a 500-word blog post based on the research findings. The post should be engaging and easy for a general audience to understand.",
       expected_output="A complete 500-word blog post about the latest AI trends.",
       agent=writer,
       context=[research_task],
   )

   # Create the Crew
   blog_creation_crew = Crew(
       agents=[researcher, writer],
       tasks=[research_task, writing_task],
       process=Process.sequential,
       llm=llm,
       verbose=True # Changed from 2 to True - verbose expects a boolean
   )

   # Execute the Crew
   print("## Running the blog creation crew with Gemini 2.0 Flash... ##")
   try:
       result = blog_creation_crew.kickoff()
       print("\n------------------\n")
       print("## Crew Final Output ##")
       print(result)
   except Exception as e:
       print(f"\nAn unexpected error occurred: {e}")

**Note**: 

The change I made in `blog_creation_crew = Crew(...)`:

- Before: 
    ```python
    verbose=2 # Set verbosity for detailed crew execution logs
    ```
- After: 
    ```python
    verbose=True # Changed from 2 to True - verbose expects a boolean
    ```

In [4]:
if __name__ == "__main__":
   main()

## Running the blog creation crew with Gemini 2.0 Flash... ##


E0000 00:00:1759936396.875284 4032657 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 9b39b02e-8102-481c-8451-d9039c44697b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Research the top 3 emerging trends in Artificial Intelligence in 2024-2025. Focus on practical           │
│  applications and potential impact.                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  As of 2024-2025, the landscape of Artificial Intelligence is witnessing several transformative trends that     │
│  are poised to redefine various industries. Here are the top three emerging trends, along with their practical  │
│  applications and potential impacts:                                                                            │
│                                                                                                                 │
│  1. **Generative AI Advancements**:                                                                             │
│     - **Key Points**: Generative AI, particularly in natural language processing and image generation, has      │
│  made remarkable strides. Tools like OpenAI's GPT models and DALL-E are being enhanced to create more nuanced,  │
│  context-aware content. These advancements are not limited to creative fields; they are reshaping sectors such  │
│  as marketing, gaming, and education. Businesses are leveraging generative models for content creation,         │
│  product design, and personalized marketing strategies.                                                         │
│     - **Practical Applications**: Companies are using generative AI for sophisticated customer service          │
│  chatbots, personalized learning experiences, and automated content generation for blogs and social media. In   │
│  product development, brands can simulate new designs or analyze customer preferences through user-generated    │
│  content.                                                                                                       │
│     - **Potential Impact**: The integration of generative AI can significantly reduce production times and      │
│  costs while enhancing creative output and user engagement. It is set to democratize content creation,          │
│  empowering smaller players in the industry to compete with established businesses more effectively.            │
│     - **Sources**: CNET, Forbes, AI Weekly                                                                      │
│                                                                                                                 │
│  2. **AI for Enhanced Cybersecurity**:                                                                          │
│     - **Key Points**: Cybersecurity is becoming a critical focus as digital threats evolve. AI tools capable    │
│  of detecting anomalies and preventing breaches in real-time are emerging. Advanced machine learning            │
│  algorithms are utilized to analyze vast amounts of data to identify potential threats and vulnerabilities      │
│  proactively.                                                                                                   │
│     - **Practical Applications**: Organizations are implementing AI systems to monitor network traffic,         │
│  analyze user behavior, and detect phishing attempts in real-time. Automated response systems are being         │
│  developed to respond to threats without human intervention, leading to quicker resolution of security          │
│  incidents.                                                                                                     │
│     - **Potential Impact**: As reliance on digital infrastructures increases, AI-enhanced cybersecurity         │
│  solutions are vital for safeguarding sensitive informa

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Content Writer                                                                                │
│                                                                                                                 │
│  Task: Write a 500-word blog post based on the research findings. The post should be engaging and easy for a    │
│  general audience to understand.                                                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Technical Content Writer                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **Exploring the Cutting-Edge Trends of Artificial Intelligence in 2024-2025**                                  │
│                                                                                                                 │
│  As we step into 2024-2025, the world of Artificial Intelligence (AI) is evolving rapidly, bringing             │
│  transformative changes that rattle not only tech industries but also reshape everyday life. From generating    │
│  captivating content to enhancing cybersecurity and personalizing user experiences, these advancements signal   │
│  a future filled with possibilities. Here are the top three trends that are at the forefront of this AI         │
│  revolution.                                                                                                    │
│                                                                                                                 │
│  ### 1. Generative AI Advancements                                                                              │
│                                                                                                                 │
│  Generative AI is the star of the show, especially in natural language processing and image generation. Tools   │
│  such as OpenAI's GPT models and DALL-E are gaining momentum, becoming more nuanced and context-aware. What     │
│  does this mean for us? Well, generative AI is not just a playground for creatives; it’s making waves in        │
│  industries like marketing, gaming, and education.                                                              │
│                                                                                                                 │
│  Companies are beginning to harness generative AI to create sophisticated customer service chatbots that        │
│  provide round-the-clock assistance, ensuring users have satisfactory experiences. In educational settings,     │
│  personalized learning experiences are becoming a reality through AI-powered platforms that adapt to students'  │
│  individual needs. Even in content creation, brands can generate automated blog posts and social media          │
│  content, effectively saving both time and effort.                                                              │
│                                                                                                                 │
│  The potential impact here is significant—it can drastically reduce production times and costs while boosting   │
│  output quality. Smaller companies, with these AI tools at their disposal, can now compete on a level playing   │
│  field against larger companies, democratizing the content creation space.                                      │
│                                                                                                                 │
│  ### 2. AI for Enhanced Cybersecurity                                                                           │
│                                                                                                                 │
│  As our reliance on digital technology grows, so does the threat of cybercrime. This is where AI comes into     │
│  play as a formidable ally in the battle for cybersecurity. Emerging AI tools, equipped with advanced machine   │
│  learning algorithms, are proving vital in detecting an

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e4468c9e-b86e-4451-8867-b517a879f9d0                                                                     │
│  Agent: Technical Content Writer                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 9b39b02e-8102-481c-8451-d9039c44697b                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: **Exploring the Cutting-Edge Trends of Artificial Intelligence in 2024-2025**                    │
│                                                                                                                 │
│  As we step into 2024-2025, the world of Artificial Intelligence (AI) is evolving rapidly, bringing             │
│  transformative changes that rattle not only tech industries but also reshape everyday life. From generating    │
│  captivating content to enhancing cybersecurity and personalizing user experiences, these advancements signal   │
│  a future filled with possibilities. Here are the top three trends that are at the forefront of this AI         │
│  revolution.                                                                                                    │
│                                                                                                                 │
│  ### 1. Generative AI Advancements                                                                              │
│                                                                                                                 │
│  Generative AI is the star of the show, especially in natural language processing and image generation. Tools   │
│  such as OpenAI's GPT models and DALL-E are gaining momentum, becoming more nuanced and context-aware. What     │
│  does this mean for us? Well, generative AI is not just a playground for creatives; it’s making waves in        │
│  industries like marketing, gaming, and education.                                                              │
│                                                                                                                 │
│  Companies are beginning to harness generative AI to create sophisticated customer service chatbots that        │
│  provide round-the-clock assistance, ensuring users have satisfactory experiences. In educational settings,     │
│  personalized learning experiences are becoming a reality through AI-powered platforms that adapt to students'  │
│  individual needs. Even in content creation, brands can generate automated blog posts and social media          │
│  content, effectively saving both time and effort.                                                              │
│                                                                                                                 │
│  The potential impact here is significant—it can drastically reduce production times and costs while boosting   │
│  output quality. Smaller companies, with these AI tools at their disposal, can now compete on a level playing   │
│  field against larger companies, democratizing the content creation space.                                      │
│                                                                                                                 │
│  ### 2. AI for Enhanced Cybersecurity                                                                           │
│                                                                                                                 │
│  As our reliance on digital technology grows, so does the threat of cybercrime. This is where AI comes into     │
│  play as a formidable ally in the battle for cybersecu


------------------

## Crew Final Output ##
**Exploring the Cutting-Edge Trends of Artificial Intelligence in 2024-2025**

As we step into 2024-2025, the world of Artificial Intelligence (AI) is evolving rapidly, bringing transformative changes that rattle not only tech industries but also reshape everyday life. From generating captivating content to enhancing cybersecurity and personalizing user experiences, these advancements signal a future filled with possibilities. Here are the top three trends that are at the forefront of this AI revolution.

### 1. Generative AI Advancements

Generative AI is the star of the show, especially in natural language processing and image generation. Tools such as OpenAI's GPT models and DALL-E are gaining momentum, becoming more nuanced and context-aware. What does this mean for us? Well, generative AI is not just a playground for creatives; it’s making waves in industries like marketing, gaming, and education.

Companies are beginning to harness gen

We will now delve into further examples within the Google ADK framework, with particular emphasis on hierarchical, parallel, and sequential coordination paradigms, alongside the implementation of an agent as an operational instrument. 